In [ ]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path

import matplotlib as mpl
import numpy as np
import pandas as pd
import requests
import xarray as xr

from oslofjord.regrid import fill_nans_with_nearest, regrid_from_norkyst

In [ ]:
OPENDAP_URL = "https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m/"
CATALOG_URL = "https://thredds.met.no/thredds/catalog/fou-hi/norkyst800m/catalog.xml"
PARAMETERS = ["temperature", "salinity", "u_eastward", "v_northward"]

### Data fetching
Download NorKyst data (ROMS model for the Norwegian coast).
This data will be used for the boundary conditions, which are set up as forcing.
Skip if you have it already.

In [ ]:
def list_opendap_files(base_url=CATALOG_URL):
    response = requests.get(base_url)

    if response.status_code != 200:
        print("Failed to retrieve the directory listing.")
        return []

    root = ET.fromstring(response.content)
    files = [
        elem.attrib["name"]
        for elem in root.findall(".//{http://www.unidata.ucar.edu/namespaces/thredds/InvCatalog/v1.0}dataset")
        if elem.attrib["name"].endswith(".nc")
    ]

    return files

In [ ]:
# names are like 20240101, so put an appropriate string to get specific files
month_string = "202404"
files = sorted([s for s in list_opendap_files() if month_string in s])
urls = [os.path.join(OPENDAP_URL, x) for x in files]
dss = [xr.open_dataset(url)[PARAMETERS] for url in urls]

In [ ]:
ds = xr.combine_by_coords(dss, combine_attrs="override")  # downloading, takes a while

In [ ]:
encoding = {var: {"zlib": True, "complevel": 5} for var in ds.data_vars}
ds.to_netcdf(
    os.path.join(Path.home(), "FjordsSim_data", "oslofjord", f"NorKyst-800m_ZDEPTHS_avg_{month_string}.nc"),
    encoding=encoding,
)

In [ ]:
del ds, dss, urls, files, encoding, month_string

### Regridding
Open the downloaded datasets and regrid them to the coordinates from a bathymetry file.

In [ ]:
FILES = sorted(
    os.path.join(Path.home(), "FjordsSim_data", "oslofjord", f)
    for f in os.listdir(os.path.join(Path.home(), "FjordsSim_data", "oslofjord"))
    if f.startswith("NorKyst-800m_ZDEPTHS_avg_")
)

In [ ]:
ds_grid = xr.open_dataset(os.path.join(Path.home(), "FjordsSim_data", "oslofjord", "bathymetry_105to232.nc"))

In [ ]:
lat_diff = np.diff(ds_grid["lat"].values)[0]
lat_faces = np.zeros(shape=ds_grid["lat"].values.shape[0] + 1)
lat_faces[0] = ds_grid["lat"].values[0] - lat_diff / 2
lat_faces[1:] = ds_grid["lat"].values + lat_diff / 2

In [ ]:
lon_diff = np.diff(ds_grid["lon"].values)[0]
lon_faces = np.zeros(shape=ds_grid["lon"].values.shape[0] + 1)
lon_faces[0] = ds_grid["lon"].values[0] - lon_diff / 2
lon_faces[1:] = ds_grid["lon"].values + lon_diff / 2

In [ ]:
z_faces = ds_grid.z_faces.values
z_centers = [(z_faces[i] + z_faces[i + 1]) / 2 for i in range(len(z_faces) - 1)]
ds_out_c = xr.Dataset(
    {
        "lat": (["lat"], ds_grid["lat"].values, {"units": "degrees_north"}),
        "lon": (["lon"], ds_grid["lon"].values, {"units": "degrees_east"}),
    }
)
ds_out_u = xr.Dataset(
    {
        "lat": (["lat"], ds_grid["lat"].values, {"units": "degrees_north"}),
        "lon": (["lon"], lon_faces, {"units": "degrees_east"}),
    }
)
ds_out_v = xr.Dataset(
    {
        "lat": (["lat"], lat_faces, {"units": "degrees_north"}),
        "lon": (["lon"], ds_grid["lon"].values, {"units": "degrees_east"}),
    }
)

In [ ]:
time_list = []
temp_list = []
salt_list = []
u_list = []
v_list = []

for file in FILES:
    ds_in = xr.open_dataset(file)
    np_time, np_temp, np_salt, np_u, np_v = regrid_from_norkyst(
        ds_in, ds_out_c, ds_out_u, ds_out_v, np.array(z_centers)
    )

    time_list.append(np_time)
    temp_list.append(np_temp)
    salt_list.append(np_salt)
    u_list.append(np_u)
    v_list.append(np_v)
    print(f"File {file} processed.")

In [ ]:
np_time = np.concatenate(time_list, axis=0)
np_temp = np.concatenate(temp_list, axis=0).astype(np.float32)
np_salt = np.concatenate(salt_list, axis=0).astype(np.float32)
np_u = np.concatenate(u_list, axis=0).astype(np.float32)
np_v = np.concatenate(v_list, axis=0).astype(np.float32)

In [ ]:
Tout_lambda = np.zeros_like(np_temp)
Sout_lambda = np.zeros_like(np_salt)
Uout_lambda = np.zeros_like(np_u)
Vout_lambda = np.zeros_like(np_v)

In [ ]:
dsout = xr.Dataset(
    {
        "T": (["time", "Nz", "Ny", "Nx"], np_temp),
        "T_lambda": (["time", "Nz", "Ny", "Nx"], Tout_lambda),
        "S": (["time", "Nz", "Ny", "Nx"], np_salt),
        "S_lambda": (["time", "Nz", "Ny", "Nx"], Sout_lambda),
        "u": (["time", "Nz", "Ny", "Nx_faces"], np_u),
        "u_lambda": (["time", "Nz", "Ny", "Nx_faces"], Uout_lambda),
        "v": (["time", "Nz", "Ny_faces", "Nx"], np_v),
        "v_lambda": (["time", "Nz", "Ny_faces", "Nx"], Vout_lambda),
    },
    coords={
        "time": np_time,
        "Nz": z_centers,
        "Nz_faces": z_faces,
        "Ny": ds_grid["lat"].values,
        "Ny_faces": lat_faces,
        "Nx": ds_grid["lon"].values,
        "Nx_faces": lon_faces,
    },
)

In [ ]:
dsout

In [ ]:
z_level = -1

In [ ]:
dsout.T.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout.S.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout.u.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout.v.isel(time=0, Nz=z_level).plot()

### Prepare the forcing dataset
add Lambda values for boundaries, add rivers, etc.

In [ ]:
ds = dsout.copy(deep=True)

In [ ]:
np_z_centers = (ds_grid.z_faces.values[:-1] + ds_grid.z_faces.values[1:]) / 2
oy, ox, oz = (value for value in ds_grid.sizes.values())
np_mask = (np_z_centers > ds_grid.h.values[..., np.newaxis]).astype(int).transpose(2, 0, 1)

In [ ]:
np_mask_u = np.zeros((oz - 1, oy, ox + 1))
np_mask_u[:, :, :-1] = np_mask
np_mask_u[:, :, 1:] = np.where(np_mask_u[:, :, 1:] == 0, np_mask, np_mask_u[:, :, 1:])

In [ ]:
np_mask_v = np.zeros((oz - 1, oy + 1, ox))
np_mask_v[:, :-1, :] = np_mask
np_mask_v[:, 1:, :] = np.where(np_mask_v[:, 1:, :] == 0, np_mask, np_mask_v[:, 1:, :])

In [ ]:
ds.T.values = fill_nans_with_nearest(ds.T) * np_mask
ds.S.values = fill_nans_with_nearest(ds.S) * np_mask
ds.u.values = fill_nans_with_nearest(ds.u) * np_mask_u
ds.v.values = fill_nans_with_nearest(ds.v) * np_mask_v

In [ ]:
ds.v.isel(time=-1, Nz=-1).plot()

There are some data gaps in the downloaded datasets

In [ ]:
new_time = pd.date_range(start=ds.time[0].values, end=ds.time[-1].values, freq="D")

In [ ]:
new_time[~new_time.isin(ds.time.values)]  # this shows which dates are missing in the original data

In [ ]:
ds = ds.reindex(time=new_time, method="ffill")

Fill in lambdas for the sourthern boundary conditions.
Lambdas that -1 < lambda < 1 are relaxation rates.
Lambdas > 2 mean flux in x direction (west to east).
Lambdas < -1 mean flux in y direction (south to north).

In [ ]:
southern_edge = 10  # use relaxation at 10 southernmost 'layers' at all depths
lambda_relaxation = 1 / (60 * 60 * 24)

In [ ]:
ds["T_lambda"][:, :, :southern_edge, :] = (lambda_relaxation) * np_mask[:, :southern_edge, :]
ds["T_lambda"] = ds["T_lambda"].astype(np.float32)
ds["S_lambda"][:, :, :southern_edge, :] = (lambda_relaxation) * np_mask[:, :southern_edge, :]
ds["S_lambda"] = ds["S_lambda"].astype(np.float32)
ds["u_lambda"][:, :, :southern_edge, :] = (lambda_relaxation) * np_mask_u[:, :southern_edge, :]
ds["u_lambda"] = ds["u_lambda"].astype(np.float32)
ds["v_lambda"][:, :, :southern_edge, :] = (lambda_relaxation) * np_mask_v[:, :southern_edge, :]
ds["v_lambda"] = ds["v_lambda"].astype(np.float32)

Add rivers

In [ ]:
# Drammenselva
ds["S"].values[:, -2:, 186, 8] = 0
ds["S_lambda"].values[:, -2:, 186, 8] = 1 / (60 * 60)
ds["v"].values[:, -2:, 187, 8] = -0.5  # negative flux means 'to the south'
ds["v_lambda"].values[:, -2:, 187, 8] = -2  # this lambda means flux in 'y' direction

In [ ]:
# Barumsbassenget
y, x = 218, 42
ds["S"].values[:, -2:, y, x] = 0
ds["S_lambda"].values[:, -2:, y, x] = 1 / (60 * 60)
ds["u"].values[:, -2:, y, x] = 0.5  # to the east
ds["u_lambda"].values[:, -2:, y, x] = 2  # x direction

In [ ]:
ds.S_lambda.isel(time=-1, Nz=-1).where(lambda x: x != 0).plot(cmap=mpl.colormaps.get_cmap("Spectral"), figsize=(7, 10))

In [ ]:
ds.S.isel(time=-1, Nz=-1).plot(cmap=mpl.colormaps.get_cmap("Spectral"), figsize=(7, 10))

In [ ]:
ds

Save

In [ ]:
encoding = {var: {"zlib": True, "complevel": 5} for var in ds.data_vars}
ds.to_netcdf(
    os.path.join(Path.home(), "FjordsSim_data", "oslofjord", "forcing_105to232.nc"),
    encoding=encoding,
)